In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'
import sys
sys.path.append('/home/iiyama/src/fastdla')
from collections.abc import Sequence
import numpy as np
try:
    import jax
    import jax.numpy as jnp
    jax.config.update('jax_enable_x64', True)
except ImportError:
    jnp = np
from qiskit.quantum_info import SparsePauliOp
from fastdla.dla import generate_dla, is_independent
from fastdla.sparse_pauli_vector import SparsePauliVector

In [2]:
def z2lgt_hva_generators(num_fermions: int) -> list[SparsePauliVector]:
    """Construct generators of the HVA for the Z2 LGT model for the given number of staggered
    fermions."""
    num_qubits = 4 * num_fermions

    generators = []

    # Mass terms Z_{n}
    for parity in [0, 1]:
        strings = ['I' * (num_qubits - isite * 2 - 1) + 'Z' + 'I' * (isite * 2)
                   for isite in range(parity, 2 * num_fermions, 2)]
        coeffs = np.ones(len(strings)) / np.sqrt(len(strings))
        generators.append(SparsePauliVector(strings, coeffs))

    # Field term X_{n,n+1}
    strings = ['I' * (num_qubits - iq - 1) + 'X' + 'I' * iq for iq in range(1, num_qubits, 2)]
    coeffs = np.ones(len(strings)) / np.sqrt(len(strings))
    generators.append(SparsePauliVector(strings, coeffs))

    # Hopping terms X_{n}Z_{n,n+1}X_{n+1} + Y_{n}Z_{n,n+1}Y_{n+1}
    for parity in [0, 1]:
        strings = []
        for isite in range(parity, 2 * num_fermions, 2):
            for site_op in ['X', 'Y']:
                paulis_reverse = ['I'] * num_qubits
                paulis_reverse[isite * 2] = site_op
                paulis_reverse[isite * 2 + 1] = 'Z'
                paulis_reverse[(isite * 2 + 2) % num_qubits] = site_op
                strings.append(''.join(paulis_reverse[::-1]))
        coeffs = np.ones(len(strings)) / np.sqrt(len(strings))
        generators.append(SparsePauliVector(strings, coeffs))

    return generators


In [3]:
def z2lgt_gauss_projector(charges: Sequence[int]) -> SparsePauliVector:
    """Construct the Gauss's law projectors for the Z2 LGT model.

    Physical states of the Z2 LGT model must be eigenstates of G_n = X_{n-1,n}Z_{n}X_{n1,n+1}. Given
    an eigenvalue for each G_n, we can construct projector to its subspace as a sum of Pauli ops.
    The overall projector will then be a product of all such projectors.
    """
    if len(charges) % 2 or not all(abs(ev) == 1 for ev in charges):
        raise ValueError('There must be an even number of charges with values +-1')

    num_fermions = len(charges) // 2
    num_qubits = 4 * num_fermions

    projector = SparsePauliVector('I' * num_qubits, 1.)
    for isite, ev in enumerate(charges):
        if ev > 0:
            # +0+ -1+ -0- +1- with +/-=1/2(I±X) and 0/1=1/2(I±Z)
            # From
            # np.sum([
            #     np.kron(np.kron([1, 1], [1, 1]), [1, 1]),
            #     np.kron(np.kron([1, -1], [1, -1]), [1, 1]),
            #     np.kron(np.kron([1, -1], [1, 1]), [1, -1]),
            #     np.kron(np.kron([1, 1], [1, -1]), [1, -1])
            # ]) / 8
            # we get array([0.5, 0, 0, 0, 0, 0, 0, 0.5]) -> 1/2(III + XZX)

            coeffs = [0.5, 0.5]
        else:
            # -1- +0- +1+ -0+ with +/-=1/2(I±X) and 0/1=1/2(I±Z)
            # From
            # np.sum([
            #     np.kron(np.kron([1, 1], [1, 1]), [1, 1]),
            #     np.kron(np.kron([1, -1], [1, -1]), [1, 1]),
            #     np.kron(np.kron([1, -1], [1, 1]), [1, -1]),
            #     np.kron(np.kron([1, 1], [1, -1]), [1, -1])
            # ]) / 8
            # we get array([0.5, 0, 0, 0, 0, 0, 0, -0.5]) -> 1/2(III - XZX)
            coeffs = [0.5, -0.5]
        strings = ['I' * num_qubits]
        paulis_reverse = ['I'] * num_qubits
        paulis_reverse[isite * 2] = 'Z'
        paulis_reverse[(isite * 2 - 1) % num_qubits] = 'X'
        paulis_reverse[(isite * 2 + 1) % num_qubits] = 'X'
        strings.append(''.join(paulis_reverse[::-1]))
        projector = projector @ SparsePauliVector(strings, coeffs)

    return projector

In [4]:
def z2lgt_u1_projector(num_fermions: int, charge: int) -> SparsePauliVector:
    """Construct the charge conservation law projector for the Z2 LGT model.

    Physical states of the Z2 LGT model must be eigenstates of Q = ∑_n Z_n where n runs over
    staggered fermions (even n: particle, odd n: antiparticle). For a fermion number F and an
    overall charge q ∈ [-2F,...,2F], we have a 2F C (F+q/2) -dimensional eigenspace.
    """
    if abs(charge) > (num_sites := 2 * num_fermions) or charge % 2 != 0:
        raise ValueError('Invalid charge value')

    # Diagonals=eigenvalues of the symmetry generator
    eigvals = np.zeros((2,) * num_sites, dtype=int)
    z = np.array([1, -1])
    for isite in range(num_sites):
        eigvals += np.expand_dims(z, tuple(range(isite)) + tuple(range(isite + 1, num_sites)))
    eigvals = eigvals.reshape(-1)
    # State indices with the given charge
    states = np.nonzero(eigvals == charge)[0]
    # Binary representations of the indices
    states_binary = (states[:, None] >> np.arange(num_sites)[None, ::-1]) % 2
    # |0><0|=1/2(I+Z), |1><1|=1/2(I-Z) -> Coefficients of I and Z for each binary digit
    # Example: [0, 1] -> [[1, 1], [1, -1]]
    states_iz = np.array([[1, 1], [1, -1]])[states_binary]
    # Take the kronecker products of the I/Z coefficients using einsum, then sum over the states to
    # arrive at the final sparse Pauli representation of the projector
    args = ()
    for isite in range(num_sites):
        args += (states_iz[:, isite], [0, isite + 1])
    args += (list(range(num_sites + 1)),)
    coeffs = np.sum(np.einsum(*args).reshape(states.shape[0], 2 ** num_sites), axis=0)
    # Take only the nonzero Paulis
    pauli_indices = np.nonzero(coeffs)[0]
    coeffs = coeffs[pauli_indices] / (2 ** num_sites)
    paulis = []
    for idx in pauli_indices:
        idx_bin = np.zeros((num_sites, 2), dtype=int)
        idx_bin[:, 1] = (idx >> np.arange(num_sites)[::-1]) % 2
        paulis.append(''.join('IZ'[i] for i in idx_bin.reshape(-1)))

    return SparsePauliVector(paulis, coeffs)

## 2 Fermions

In [13]:
num_fermions = 2
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1]
charge = 0

In [14]:
# Full HVA generators
generators_full = z2lgt_hva_generators(num_fermions)

In [15]:
generators_full

[SparsePauliVector(['IIIIIIIZ', 'IIIZIIII'], array([0.70710678+0.j, 0.70710678+0.j])),
 SparsePauliVector(['IIIIIZII', 'IZIIIIII'], array([0.70710678+0.j, 0.70710678+0.j])),
 SparsePauliVector(['IIIIIIXI', 'IIIIXIII', 'IIXIIIII', 'XIIIIIII'], array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j])),
 SparsePauliVector(['IIIIIXZX', 'IIIIIYZY', 'IXZXIIII', 'IYZYIIII'], array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j])),
 SparsePauliVector(['IIIXZXII', 'IIIYZYII', 'ZXIIIIIX', 'ZYIIIIIY'], array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j]))]

In [17]:
# Compute the DLA of the full space
dla_full = generate_dla(generators_full, min_tasks=100, max_workers=54, verbosity=3)
print(f'DLA dimension is {len(dla_full)}')

Starting with 10 commutators..
Evaluating 10/10 commutators for independence
Sorted in 0.00s
Processing SPV 0/10 (DLA dim 5)..
Found 5 new ops in 0.00s
Adding 35 commutators; total 35
Current DLA dimension: 10
Outer loop took 0.01s
Evaluating 35/35 commutators for independence
Sorted in 0.00s
Processing SPV 0/35 (DLA dim 10)..
Found 16 new ops in 0.01s
Adding 280 commutators; total 280
Current DLA dimension: 26
Outer loop took 0.04s
Evaluating 256/280 commutators for independence
Sorted in 0.00s
Processing SPV 0/256 (DLA dim 26)..
Found 97 new ops in 0.12s
Adding 7178 commutators; total 7202
Current DLA dimension: 123
Outer loop took 0.97s
Evaluating 966/7202 commutators for independence
Sorted in 0.01s
Processing SPV 0/966 (DLA dim 123)..
Processing SPV 500/966 (DLA dim 220)..
Found 165 new ops in 1.46s
Adding 33825 commutators; total 40061
Current DLA dimension: 288
Outer loop took 8.62s
Evaluating 37629/40061 commutators for independence
Sorted in 0.66s
Processing SPV 0/37629 (DLA d

In [18]:
# Projectors
gauss_projector = z2lgt_gauss_projector(gauss_eigvals)
u1_projector = z2lgt_u1_projector(num_fermions, charge)
symm_projector = gauss_projector @ u1_projector
# HVA generators and symmetry generators are simultaneously diagonalizable
# -> HVA generators can be projected onto the symmetry subspace by one-side application of the
# projectors. The resulting operators are the HVA generators in the subspace.
generators = [(op @ symm_projector).normalize() for op in generators_full]

In [19]:
# Check the dimensionality of the symmetry subspace
# dense_matrix = symm_projector.to_matrix(npmod=jnp)
dense_matrix = SparsePauliOp(symm_projector.paulis, symm_projector.coeffs).to_matrix()
proj_eigvals = jnp.linalg.eigh(dense_matrix).eigenvalues
print(f'Hilbert subspace dimension is {np.nonzero(np.isclose(proj_eigvals, 1.))[0].shape[0]}')

Hilbert subspace dimension is 12


In [20]:
# Refine the generators list in case some are linearly dependent in the subspace
generators_indep = [generators[0]]
for gen in generators[1:]:
    if is_independent(gen, generators_indep):
        generators_indep.append(gen)
print(f'{len(generators_indep)} generators are independent')

4 generators are independent


In [21]:
# Compute the DLA of the subspace
dla = generate_dla(generators_indep, verbosity=3, min_tasks=100, max_workers=54)
print(f'Subspace DLA dimension is {len(dla)}')

Starting with 6 commutators..
Evaluating 6/6 commutators for independence
Sorted in 0.00s
Processing SPV 0/6 (DLA dim 4)..
Found 5 new ops in 0.00s
Adding 30 commutators; total 30
Current DLA dimension: 9
Outer loop took 0.01s
Evaluating 30/30 commutators for independence
Sorted in 0.00s
Processing SPV 0/30 (DLA dim 9)..
Found 16 new ops in 0.01s
Adding 264 commutators; total 264
Current DLA dimension: 25
Outer loop took 0.05s
Evaluating 228/264 commutators for independence
Sorted in 0.01s
Processing SPV 0/228 (DLA dim 25)..
Found 35 new ops in 0.03s
Adding 1470 commutators; total 1506
Current DLA dimension: 60
Outer loop took 0.18s
Evaluating 609/1506 commutators for independence
Sorted in 0.04s
Processing SPV 0/609 (DLA dim 60)..
Processing SPV 500/609 (DLA dim 63)..
Found 4 new ops in 0.15s
Adding 246 commutators; total 1143
Current DLA dimension: 64
Outer loop took 0.21s
Evaluating 653/1143 commutators for independence
Sorted in 0.04s
Processing SPV 0/653 (DLA dim 64)..
Processing 

## 3 Fermions

In [5]:
num_fermions = 3
# Determine the charge sector (symmetry subspace) to investigate
gauss_eigvals = [1, -1, 1, -1, 1, -1]
charge = 0

In [6]:
# Full HVA generators
generators_full = z2lgt_hva_generators(num_fermions)

In [7]:
generators_full

[SparsePauliVector(['IIIIIIIIIIIZ', 'IIIIIIIZIIII', 'IIIZIIIIIIII'], array([0.57735027+0.j, 0.57735027+0.j, 0.57735027+0.j])),
 SparsePauliVector(['IIIIIIIIIZII', 'IIIIIZIIIIII', 'IZIIIIIIIIII'], array([0.57735027+0.j, 0.57735027+0.j, 0.57735027+0.j])),
 SparsePauliVector(['IIIIIIIIIIXI', 'IIIIIIIIXIII', 'IIIIIIXIIIII', 'IIIIXIIIIIII', 'IIXIIIIIIIII', 'XIIIIIIIIIII'], array([0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j,
        0.40824829+0.j, 0.40824829+0.j])),
 SparsePauliVector(['IIIIIIIIIXZX', 'IIIIIIIIIYZY', 'IIIIIXZXIIII', 'IIIIIYZYIIII', 'IXZXIIIIIIII', 'IYZYIIIIIIII'], array([0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j,
        0.40824829+0.j, 0.40824829+0.j])),
 SparsePauliVector(['IIIIIIIXZXII', 'IIIIIIIYZYII', 'IIIXZXIIIIII', 'IIIYZYIIIIII', 'ZXIIIIIIIIIX', 'ZYIIIIIIIIIY'], array([0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j, 0.40824829+0.j,
        0.40824829+0.j, 0.40824829+0.j]))]

In [8]:
# Compute the DLA of the full space
# dla_full = generate_dla(generators_full, min_tasks=100, max_workers=54, verbosity=3)
# print(f'DLA dimension is {len(dla_full)}')

In [9]:
# Projectors
gauss_projector = z2lgt_gauss_projector(gauss_eigvals)
u1_projector = z2lgt_u1_projector(num_fermions, charge)
symm_projector = gauss_projector @ u1_projector
# HVA generators and symmetry generators are simultaneously diagonalizable
# -> HVA generators can be projected onto the symmetry subspace by one-side application of the
# projectors. The resulting operators are the HVA generators in the subspace.
generators = [(op @ symm_projector).normalize() for op in generators_full]

In [10]:
# Check the dimensionality of the symmetry subspace
# dense_matrix = symm_projector.to_matrix(npmod=jnp)
dense_matrix = SparsePauliOp(symm_projector.paulis, symm_projector.coeffs).to_matrix()
proj_eigvals = jnp.linalg.eigh(dense_matrix).eigenvalues
print(f'Hilbert subspace dimension is {np.nonzero(np.isclose(proj_eigvals, 1.))[0].shape[0]}')

Hilbert subspace dimension is 40


In [11]:
# Refine the generators list in case some are linearly dependent in the subspace
generators_indep = [generators[0]]
for gen in generators[1:]:
    if is_independent(gen, generators_indep):
        generators_indep.append(gen)
print(f'{len(generators_indep)} generators are independent')

4 generators are independent


In [12]:
# Compute the DLA of the subspace
dla = generate_dla(generators_indep, verbosity=3, min_tasks=100, max_workers=54)
print(f'Subspace DLA dimension is {len(dla)}')

Starting with 6 commutators..
Evaluating 6/6 commutators for independence
Sorted in 0.00s
Processing SPV 0/6 (DLA dim 4)..
Found 5 new ops in 4.28s
Adding 30 commutators; total 30
Current DLA dimension: 9
Outer loop took 4.30s
Evaluating 30/30 commutators for independence
Sorted in 0.00s
Processing SPV 0/30 (DLA dim 9)..
Found 16 new ops in 0.01s
Adding 264 commutators; total 264
Current DLA dimension: 25
Outer loop took 0.05s
Evaluating 115/264 commutators for independence
Sorted in 1.06s
Processing SPV 0/115 (DLA dim 25)..
Found 50 new ops in 1.31s
Adding 2475 commutators; total 2624
Current DLA dimension: 75
Outer loop took 2.68s
Evaluating 108/2624 commutators for independence
Sorted in 1.89s
Processing SPV 0/108 (DLA dim 75)..
Found 45 new ops in 6.54s
Adding 4365 commutators; total 6881
Current DLA dimension: 120
Outer loop took 8.79s
Evaluating 106/6881 commutators for independence
Sorted in 0.97s
Processing SPV 0/106 (DLA dim 120)..
Found 64 new ops in 8.89s
Adding 9696 commuta